# Lab 5 data

This notebook takes the raw Lab 5 data and outputs cleaned data and graphs

**Inputs**: `PHY-202L\data\raw\Lab_5.csv`

**Outputs**: `PHY-202L\data\processed\Lab_5_clean.csv`

# 0) cfg

Configuration/parameters setup for notebook

In [ ]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)

# 1) Data Wrangler export

This cell reads the raw CSV and applies transformations

**Outputs**: `df_clean`

In [ ]:
import pandas as pd


def clean_data(df):
    # Change column type to float64 for columns: 'Mu (0)', 'N (revs)' and 5 other columns
    df = df.astype(
        {
            "Mu (0)": "float64",
            "N (revs)": "float64",
            "I (A)": "float64",
            "L (m)": "float64",
            "R (m)": "float64",
            "z (m)": "float64",
            "Magnetic Field (Measured) (mT)": "float64",
        }
    )
    return df


df = pd.read_csv("../data/raw/Lab_5.csv")

df_clean = clean_data(df.copy())
df_clean.head()

# 2) Derived columns

Add columns needed to complete relevant calculations and graphs

**Column additions:**
- **Magnetic Field (Lab Manual Theoretical) (T)**  
  $$
  B_z(z) = \frac{\mu_0\,N\,I\,R^2}{2\,\bigl(z^2 + R^2\bigr)^{3/2}}
  $$

- **Magnetic Field (Textbook Theoretical) (T)**  
  $$
  B_z(z) = \frac{\mu_0\,n\,I}{2}\!\left(
  \frac{z+\tfrac{L}{2}}{\sqrt{(z+\tfrac{L}{2})^2 + R^2}}
  - \frac{z-\tfrac{L}{2}}{\sqrt{(z-\tfrac{L}{2})^2 + R^2}}
  \right),\qquad n=\frac{N}{L}
  $$

## % Error

- **vs Textbook (finite solenoid):**
$$
\%\ \mathrm{error}_{\text{textbook}}(z_i)=
\left|
\frac{B_{\mathrm{meas}}(z_i)-B_{\mathrm{theory}}^{\mathrm{text}}(z_i)}
{B_{\mathrm{theory}}^{\mathrm{text}}(z_i)}
\right|\times 100\%.
$$

- **vs Lab Manual (single loop):**
$$
\%\ \mathrm{error}_{\text{lab}}(z_i)=
\left|
\frac{B_{\mathrm{meas}}(z_i)-B_{\mathrm{theory}}^{\mathrm{lab}}(z_i)}
{B_{\mathrm{theory}}^{\mathrm{lab}}(z_i)}
\right|\times 100\%.
$$

In [ ]:
import numpy as np

mu0 = 4 * np.pi * 1e-7  # H/m

# Pull columns (falls back to constants if a column isn't present)
z = df_clean["z (m)"]  # must exist
R = df_clean["R (m)"]  # must exist
N = df_clean["N (revs)"] if "N (revs)" in df_clean.columns else 400
I = df_clean["I (A)"] if "I (A)" in df_clean.columns else 0.3
L = df_clean["L (m)"] if "L (m)" in df_clean.columns else 0.04  # solenoid length

# Convert measured mT → T
df_clean["Magnetic Field (Measured) (T)"] = df_clean["Magnetic Field (Measured) (mT)"] / 1000.0

# -------------------------------
# Lab Manual Theoretical (coil)
# Bz = mu0*N*I*R^2 / [ 2*(z^2 + R^2)^(3/2) ]
# -------------------------------
df_clean["Magnetic Field (Lab Manual Theoretical) (T)"] = (
    mu0 * N * I * R**2 / (2.0 * (z**2 + R**2) ** 1.5)
)


# -------------------------------
# Textbook Theoretical (finite solenoid, centered at z=0)
# Bz = (mu0 * n * I / 2) * [ (z+L/2)/sqrt((z+L/2)^2 + R^2) - (z-L/2)/sqrt((z-L/2)^2 + R^2) ]
# where n = N/L
# Works with scalars or Series for N, I, L, R, z
# -------------------------------
def B_solenoid_on_axis(z, N, I, L, R, mu0=mu0):
    n = N / L
    z1, z2 = z + L / 2.0, z - L / 2.0
    return (mu0 * n * I / 2.0) * (z1 / np.sqrt(z1**2 + R**2) - z2 / np.sqrt(z2**2 + R**2))


df_clean["Magnetic Field (Textbook Theoretical) (T)"] = B_solenoid_on_axis(z, N, I, L, R)

# -------------------------------
# % Error between Measured and Lab Manual Theoretical
# -------------------------------
df_clean["Relative Error (Lab Manual)"] = 100.0 * abs(
    (
        df_clean["Magnetic Field (Measured) (T)"]
        - df_clean["Magnetic Field (Lab Manual Theoretical) (T)"]
    )
    / df_clean["Magnetic Field (Lab Manual Theoretical) (T)"]
)

# -------------------------------
# % Error between Measured and Textbook Theoretical
# -------------------------------
df_clean["Relative Error (Textbook)"] = 100.0 * abs(
    (
        df_clean["Magnetic Field (Measured) (T)"]
        - df_clean["Magnetic Field (Textbook Theoretical) (T)"]
    )
    / df_clean["Magnetic Field (Textbook Theoretical) (T)"]
)

df_clean.head()

# 3) Save the processed dataset to file

Take all of the transformations to the original CSV done in this file and materialize them into a final file

**Outputs**: `Lab _5_clean.csv`

In [ ]:
out_path = "../data/processed/Lab_5_clean.csv"
df_clean.to_csv(out_path, index=False)
out_path

# 4) Quick visuals

Various figures using plotly and using matplotlib.

*Interactive plotly figure*:

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

dpf = df_clean.copy()

dpf = dpf.sort_values("z (m)")

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=dpf["z (m)"],
        y=dpf["Magnetic Field (Measured) (T)"],
        mode="markers+lines",
        name="Measured",
        line=dict(color="blue", width=2),
        marker=dict(size=6),
    )
)

fig.add_trace(
    go.Scatter(
        x=dpf["z (m)"],
        y=dpf["Magnetic Field (Lab Manual Theoretical) (T)"],
        mode="markers+lines",
        name="Lab Manual Theoretical",
        line=dict(color="orange", width=2, dash="dash"),
        marker=dict(size=6),
    )
)

fig.add_trace(
    go.Scatter(
        x=dpf["z (m)"],
        y=dpf["Magnetic Field (Textbook Theoretical) (T)"],
        mode="markers+lines",
        name="Textbook Theoretical",
        line=dict(color="green", width=2, dash="dot"),
        marker=dict(size=6),
    )
)

fig.update_layout(
    title="Magnetic Field vs. Position Along Axis of Solenoid",
    xaxis_title="Position z (m)",
    yaxis_title="Magnetic Field B (T)",
    legend_title="Legend",
    template="plotly_white",
)

fig.show()

*matplotlib figure*:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

dpf = df_clean.copy()

dpf = dpf.sort_values("z (m)")

fix, ax = plt.subplots()

ax.scatter(dpf["z (m)"], dpf["Magnetic Field (Measured) (T)"], label="Measured", color="black")
ax.plot(
    dpf["z (m)"],
    dpf["Magnetic Field (Lab Manual Theoretical) (T)"],
    label="Lab Manual Theoretical",
    color="red",
)
ax.plot(
    dpf["z (m)"],
    dpf["Magnetic Field (Textbook Theoretical) (T)"],
    label="Textbook Theoretical",
    color="blue",
)

ax.set_title("Magnetic Field vs. Distance (on-axis)")
ax.set_xlabel("Distance (m)")
ax.set_ylabel("Magnetic Field (T)")
ax.grid(True)
ax.legend()

plt.show()